# Take Your Analysis to the Next Level with Interactive Dashboarding Libraries
##### and make it available to everyone!  


**Mehmet Ergene, [@Cyb3rMonk](https://twitter.com/Cyb3rMonk)**  

Security Researcher & Data Scientist, Binalyze

---

# $whoami



* Microsoft Security MVP
* MSTICPy council member
* Blog: https://posts.bluraven.io
* GitHub: https://github.com/Cyb3r-Monk
* Non-tech:
  * Lindy hopper
  * Handpan player
  
---

# Agenda
* Interactive Dashboarding Libraries
* hvPlot Demo
* Panel Demo
* Conclusion

---

## Interactive Dashboarding Libraries

* Make data analysis interactive

* Enable developing sharable data apps

* Enable end users to perform data analysis

  * No Python, Jupyter or data analysis skills required! 

## Overview of Libraries

|Dash                              |Streamlit              |**Panel**            |Voila                       |
|----------------------------------|-----------------------|---------------------|----------------------------|
|Medium effort                     |Low effort             |**Low effort**       |Low effort                  |
|Jupyter support with jupyter-dash |No Jupyter support     |**Supports Jupyter** |Single-page apps            |
|Requires front-end engineers      |Mostly single-page apps|**Multi-page apps**  |Requires front-end engineers|


# Plotting with hvPlot
**A familiar and high-level API for data exploration and visualization**  

![image](https://hvplot.holoviz.org/assets/diagram.svg )


## Getting the Data

In [34]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [35]:
df = pd.read_pickle('sampledata.pkl')

In [36]:
df.head()

,TimeGenerated,MachineName,AppName,AppCategory,AppScore,TotalMBytes
0,2022-11-26 22:06:35.945300100,workstation02,Office Portal,Collaboration,10,0.035
1,2022-11-26 22:06:35.945300100,workstation01,CloudFlare,Security,7,0.730
2,2022-11-26 22:06:35.945300100,workstation01,Amazon CloudFront,Hosting services,10,0.478
3,2022-11-26 22:06:35.945300100,workstation02,Microsoft Online Services,IT services,10,0.338
4,2022-11-26 22:06:35.945300100,workstation01,Microsoft Online Services,IT services,10,0.677


In [ ]:
# because of a bug, the order of import is important(for now)
import panel as pn
pn.extension('tabulator')
import hvplot.pandas # this adds hvplot plotting methods to DataFrame

## Quick Exploration with hvplot.explorer()
Explorer is a Panel-based web application with which you can easily explore your data.

In [37]:
hvexplorer = hvplot.explorer(df, width=800) # it can take arguments as well. 
hvexplorer

hvDataFrameExplorer(axes=Axes(explorer=..., height=None, legend='right', logx=False, logy=False, name='Axes11004', responsive=False, shared_axes=True, width=800, xlim=None, ylim=None), by=[], colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping11034', rescale_discrete_levels=True, symmetric=False), groupby=[], kind='line', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels11055', rot=0, title='', xlabel='', ylabel=''), name='hvDataFrameExplorer10989', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations11075', rasterize=False, x_sampling=None, y_sampling=None), style=Style(alpha=1, explorer=..., name='Style11095'), x='index', y='TimeGenerated', y_multi=[], z=None)

### Creating Plots

In [38]:
df.hvplot.scatter(x='TimeGenerated', y='TotalMBytes', by='AppScore', groupby='MachineName', height=400, responsive=True, widget_location='right_top')
# we can also use df.hvplot(kind='scatter') syntax
# df.hvplot(kind='scatter, x='TimeGenerated', y='TotalMBytes', by='AppCategory', groupby='MachineName')

Row
    [0] HoloViews(DynamicMap)
    [1] Column
        [0] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='MachineName', options=['workstation02', ...], value='workstation02', width=250)
        [1] VSpacer()

### Combining Plots
* Layout
* Overlay

In [39]:
wks01_plot = df.query('MachineName == "workstation01" & TotalMBytes > 5').hvplot.scatter(title='workstation01', x='TimeGenerated', y='TotalMBytes', by='AppScore')
wks02_plot = df.query('MachineName == "workstation02" & TotalMBytes > 5').hvplot.scatter(title='workstation02', x='TimeGenerated', y='TotalMBytes', by='AppScore')

layout_plot = wks01_plot + wks02_plot

layout_plot

:Layout
   .NdOverlay.I  :NdOverlay   [AppScore]
      :Scatter   [TimeGenerated]   (TotalMBytes)
   .NdOverlay.II :NdOverlay   [AppScore]
      :Scatter   [TimeGenerated]   (TotalMBytes)

In [40]:
# Overlay
overlay_plot = wks01_plot * wks02_plot

overlay_plot.options(title='workstation01 & workstation02 overlay')

:Overlay
   .NdOverlay.I  :NdOverlay   [AppScore]
      :Scatter   [TimeGenerated]   (TotalMBytes)
   .NdOverlay.II :NdOverlay   [AppScore]
      :Scatter   [TimeGenerated]   (TotalMBytes)

## Analysis with Interactive Pipelines
hvPlot isn’t only a plotting library, it is dedicated to make data exploration easier.  
When analysing data, we often need to change parameteres and re-run the cells. This may be quite cumbersome. 
By replacing parameters with widgets and binding them to our pipeline, we can get full interactive control. 

### Define Widgets

In [41]:
machines = list(df['MachineName'].unique())
machines

['workstation02', 'workstation01', 'workstation03', 'workstation04']

In [42]:
# create a widget for selecting machines
machine_selector = pn.widgets.Select(options = machines, name = 'Machine')

In [43]:
machine_selector

Select(name='Machine', options=['workstation02', ...], value='workstation02')

In [44]:
apps = list(df['AppName'].unique())
app_selector = pn.widgets.Select(options = apps, name = 'Apps')
app_selector

Select(name='Apps', options=['Office Portal', ...], value='Office Portal')

### Define a Function

In [45]:
def df_filter_func(machine, app):
    filter_mask = (df['MachineName'] == machine) & (df['AppName'] == app)
    filtered_df = df.loc[filter_mask, :]
    return filtered_df

### Define Interactive hvPLot Object
Bind the function and widgets to the interactive object

In [46]:
df_interactive = hvplot.bind(df_filter_func, machine_selector, app_selector).interactive(width = 800)
df_interactive.head(10)

In [47]:
type(df_interactive)

hvplot.interactive.Interactive

In [ ]:
# all pandas methods are still available on the interactive object
df.interactive.

### Add More Interactivity and Plotting
Filter data by date range and plot it

In [48]:
dt_range_widget = pn.widgets.DateRangeSlider(start=df.TimeGenerated.min(), end=df.TimeGenerated.max(), width=600) # we can specify values for star tand end as well

# add widget values as a filter in the interactive hvplot object
filter_mask = (df_interactive.TimeGenerated >= dt_range_widget.param.value_start) & (df_interactive.TimeGenerated <= dt_range_widget.param.value_end)

# filter the interactive object and plot the filtered data with hvplot accessor.(we already imported hvplot.pandas previously)
interactive_plot = df_interactive[filter_mask].hvplot.scatter(x='TimeGenerated', y='TotalMBytes', grid=True, title=app_selector)
interactive_plot

# Interactive Dashboarding with Panel
Panel is a high-level app and dashboarding solution for Python. It provides a wide range of components for easily composing panels, apps, and dashboards both in the notebook and as standalone apps. The components can be broken down into three broad classes of objects:

- `Pane` wraps a user supplied object of almost any type and turns it into a renderable view. When the wrapped object or any parameter changes, a pane will update the view accordingly.
- `Widget` object is a control component that allows users to provide input to your app or dashboard, typically by clicking or editing objects in a browser, but also controllable from within Python.
- `Panel` is a hierarchical container to lay out multiple components (panes, widgets, or other Panels) into an arrangement that forms an app or dashboard.

In [ ]:
# we already imported panel and hvplot above
# import panel as pn 
# import hvplot.pandas

# pn.extension() # setting the sizing mode for panels/widgets
pn.extension('tabulator', sizing_mode='stretch_width') # we need to load the extension that we want to use when using Jupyter Notebook

### Tabulator
Tabulator is a widget that allows displaying and editing a pandas DataFrame.

In [49]:
df['Comment'] = '' # Adding a new column for commenting feature
df.head(5)

,TimeGenerated,MachineName,AppName,AppCategory,AppScore,TotalMBytes,Comment
0,2022-11-26 22:06:35.945300100,workstation02,Office Portal,Collaboration,10,0.035,
1,2022-11-26 22:06:35.945300100,workstation01,CloudFlare,Security,7,0.730,
2,2022-11-26 22:06:35.945300100,workstation01,Amazon CloudFront,Hosting services,10,0.478,
3,2022-11-26 22:06:35.945300100,workstation02,Microsoft Online Services,IT services,10,0.338,
4,2022-11-26 22:06:35.945300100,workstation01,Microsoft Online Services,IT services,10,0.677,


In [50]:
# create a tabulator for interactively vieweing the data, filtering and so on
# make rows selectable with a checkbox
# set some columns non-editable except the Comment column
# add header filters
# add additional configuration
df_tabulator = pn.widgets.Tabulator(df,
                                    show_index=False, 
                                    page_size=15,
                                    sizing_mode='stretch_width', 
                                    layout='fit_data', 
                                    selectable='checkbox', 
                                    editors={
                                        'TimeGenerated': None,
                                        'MachineName': None,
                                        'AppName': None,
                                        'Comment': 'input'
                                    }, 
                                    header_filters={
                                        'MachineName': {'type': 'input', 'func': 'like', 'placeholder': 'Enter Machine'},
                                        'AppName': {'type': 'input', 'func': 'like', 'placeholder': 'Enter App'},
                                        'TotalMBytes': {'type': 'number', 'func': '>=', 'placeholder': 'Enter minimum KB'}
                                    },
                                    configuration={'resizable': True,
                                                   'headerSort': True,
                                                   'selectablePersistence': True
                                                  }
                                   )

df_tabulator

Tabulator(editors={'TimeGenerated': None, ...}, header_filters={'MachineName': {'type': '...}, layout='fit_data', page_size=15, pagination='local', selectable='checkbox', show_index=False, sizing_mode='stretch_width', value=              ...)

In [51]:
df_tabulator.selected_dataframe

,TimeGenerated,MachineName,AppName,AppCategory,AppScore,TotalMBytes,Comment
4,2022-11-26 22:06:35.945299968,workstation01,Microsoft Online Services,IT services,10,0.677,
5,2022-11-26 22:06:35.945299968,workstation01,Microsoft Teams,Online meetings,10,0.260,comment
6,2022-11-26 22:06:35.945299968,workstation01,Amazon Web Services,Cloud computing platform,10,0.153,


In [52]:
# create a date range slider based on the TimeGenerated values of `df`
# set start and end values
# set default values when the slider is called
# set step in milliseconds (it is applied when moving the slider)
dt_range_slider = pn.widgets.DatetimeRangeSlider(sizing_mode='stretch_width',
    name='Date Range Slider',
    start=df['TimeGenerated'].min(), end=df['TimeGenerated'].max(),
    value=(df['TimeGenerated'].min(), df['TimeGenerated'].min() + pd.Timedelta(days=1)),
    step = 1800000
)

dt_range_slider

DatetimeRangeSlider(end=Timestamp('2022-11-29 0..., name='Date Range Slider', sizing_mode='stretch_width', start=Timestamp('2022-11-26 1..., step=1800000, value=(Timestamp('2022-11-26 19:..., value_end=Timestamp('2022-11-27 1..., value_start=Timestamp('2022-11-26 1...)

In [53]:
machines = sorted(df['MachineName'].unique())
machine_selector = pn.widgets.MultiSelect(options=machines, value=machines[:], name='Machine Filter', sizing_mode=None)
machine_selector

MultiSelect(name='Machine Filter', options=['workstation01', ...], value=['workstation01', ...])

In [54]:
# Bind widgets to tabulator as filters for correspoinding columns

df_tabulator.add_filter(dt_range_slider, 'TimeGenerated')
df_tabulator.add_filter(machine_selector, 'MachineName')

In [55]:
# put all the pieces together
my_dashboard = pn.Column(dt_range_slider, pn.Row(machine_selector, df_tabulator), interactive_plot)

my_dashboard

Column
    [0] DatetimeRangeSlider(end=Timestamp('2022-11-29 0..., name='Date Range Slider', sizing_mode='stretch_width', start=Timestamp('2022-11-26 1..., step=1800000, value=(Timestamp('2022-11-26 19:..., value_end=Timestamp('2022-11-27 1..., value_start=Timestamp('2022-11-26 1...)
    [1] Row
        [0] MultiSelect(name='Machine Filter', options=['workstation01', ...], value=['workstation01', ...])
        [1] Tabulator(editors={'TimeGenerated': None, ...}, header_filters={'MachineName': {'type': '...}, layout='fit_data', page_size=15, pagination='local', selectable='checkbox', show_index=False, sizing_mode='stretch_width', value=              ...)
    [2] Interactive(Interactive)

In [56]:
#df_tabulator = pn.widgets.Tabulator(srum_net_usage_df, show_index=False, sizing_mode='stretch_width', layout='fit_data', selectable='checkbox')

def click_fn(event):
    text = f'Clicked cell in {event.column!r} column, row {event.row!r} with value {event.value!r}'
    print(text)
    

df_tabulator.on_click(click_fn)
# df_tabulator.on_edit(lambda e: print(e.column, e.row, e.old, e.value))

In [57]:
my_dashboard

Clicked cell in 'MachineName' column, row 1382 with value 'workstation02'

Column
    [0] DatetimeRangeSlider(end=Timestamp('2022-11-29 0..., name='Date Range Slider', sizing_mode='stretch_width', start=Timestamp('2022-11-26 1..., step=1800000, value=(datetime.datetime(2022, ..., value_end=datetime.datetime(2022, ..., value_start=datetime.datetime(2022, ...)
    [1] Row
        [0] MultiSelect(name='Machine Filter', options=['workstation01', ...], value=['workstation02', ...])
        [1] Tabulator(editors={'TimeGenerated': None, ...}, header_filters={'MachineName': {'type': '...}, layout='fit_data', page=2, page_size=15, pagination='local', selectable='checkbox', show_index=False, sizing_mode='stretch_width', value=              ...)
    [2] Interactive(Interactive)